# Basic Text Regressor using Auto-Keras

Basic implementatio for toxic comment challange.

In [ ]:
#test='test'
test='varza'
sample=0.05

In [ ]:
import sys
package_dir = '../input/autokeras-april-2021'
sys.path.insert(0, package_dir)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns, numpy as np

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import rankdata

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

import autokeras as ak

Load data

In [ ]:
pseudoscores_df = pd.read_csv("../input/pseudoscores-jigsaw-toxic-comments/validation_data_pseudo_scores_clasament.csv",index_col=0)
#https://www.kaggle.com/crischir/auto-keras-text-regression-cpu/edit/run/85744222
if test =='test':
    pseudoscores_df=pseudoscores_df.sample(frac=sample)

In [ ]:
pseudoscores_df['score']=pseudoscores_df['score']+2.90

In [ ]:
pseudoscores_df.head(1)

In [ ]:
df = pd.read_csv("../input/jigsaw-score-augumentation/jigsaw_train_hate_annotationprob.csv",index_col=0)
if test =='test':
    df=df.sample(frac=sample)

In [ ]:
df.describe()

In [ ]:
ruddit_df = pd.read_csv("../input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv",index_col=0)
if test =='test':
    ruddit_df=ruddit_df.sample(frac=sample)

In [ ]:
ruddit_df.head()

In [ ]:
ruddit_df.describe()

In [ ]:
df['target']=df['proposed_score3']+0.1*df['offensive_agreement_rating']

In [ ]:
df["score_align"]=(df['target']+0.0667)*(20.597967/5.783141)


In [ ]:
df['target']=df["score_align"]

In [ ]:
df= df[['text', 'target']]

In [ ]:
pseudoscores_df.rename(columns={"score": 'target'}, inplace = True)

In [ ]:
df1=pseudoscores_df[['text', 'target']]

In [ ]:
mermeleala_df=df.append(df1)

In [ ]:
mermeleala_df.describe()

In [ ]:
mermeleala_df.tail()

In [ ]:
len(mermeleala_df)

In [ ]:
x = df['target']
ax = sns.displot(x, height=6, aspect=4, kde=True)
plt.figure(figsize=(5,40))
plt.show()

In [ ]:
#df["target"]=df['target']+0.066618

In [ ]:
# ruddit_df['offensiveness_score']=ruddit_df['offensiveness_score']+0.889000

# ruddit_df['offensiveness_score']=ruddit_df['offensiveness_score']*3.1091

In [ ]:
ruddit_df.rename(columns={'txt':'text','offensiveness_score':'target'},inplace=True)


In [ ]:

ruddit_df= ruddit_df[['text', 'target']]

Train test split. Not used/usefull. 

In [ ]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.10)
#valid, test = train_test_split(valid, test_size=0.3)

In [ ]:
df_test=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv",index_col=0)
if test =='test':
    df_test=df_test.sample(frac=sample)

In [ ]:
comments_to_score=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv",index_col=0)
if test =='test':
    comments_to_score=comments_to_score.sample(frac=sample)

In [ ]:
X_less_toxic =df_test.less_toxic.values
X_more_toxic =df_test.more_toxic.values
X_comments_to_score =comments_to_score.text.values

In [ ]:
train_data = train['text'].values
train_target = train['target'].values

valid_data = valid['text'].values
valid_target = valid['target'].values

print(train_data.shape, train_target.shape)
print(valid_data.shape, valid_target.shape)

Simple model 0.6745383286834064 

* validation score with second conv block and 3 epoch:
0.6651720472963997


Text regressor

In [ ]:
# Initialize the text regressor.
#reg = ak.TextRegressor(overwrite=True, max_trials=2,directory='/kaggle/working/models',loss='MeanAbsolutePercentageError',tuner='hyperband')  # It tries 10 different models.
reg = ak.TextRegressor(overwrite=True, max_trials=2,directory='/kaggle/working/models',loss='MeanAbsolutePercentageError')  # It tries 10 different models.
# Feed the text regressor with training data.
reg.fit(train_data, train_target, epochs=4)
# Predict with the best model.
predicted_y = reg.predict(valid_data)
# Evaluate the best model with testing data.
print(reg.evaluate(valid_data, valid_target))

In [ ]:
#preds = reg.predict(X_less_toxic).squeeze().tolist()
#p1 = predictor.predict(X_less_toxic)
p1 = reg.predict(X_less_toxic).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p2 = reg.predict(X_more_toxic).squeeze().tolist()#help(reg)

In [ ]:
def compare(p1,p2):
    z=0
    k=0
    n=len(p1)
    for i in range(len(p1)):
        if p1[i]< p2[i]:
            z=z+1
        else:
            k=k+1
            #print('nu')
    print(z/n)
    return z/n

In [ ]:
# Validation Accuracy
compare(p1,p2)

In [ ]:
scored_sub=reg.predict(X_comments_to_score).squeeze().tolist()

In [ ]:
comments_to_score['score1_rank']=rankdata( scored_sub, method='ordinal') 

In [ ]:
comments_to_score['score1_df']=scored_sub

In [ ]:
comments_to_score.head()

Prepare a keras model. 

In [ ]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
# Use separable Conv layers in Keras.
output_node = ak.ConvBlock(separable=True)(output_node)
#output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=3
)
reg.fit(train_data, train_target, epochs=7,validation_split=0.15)

Validate results

In [ ]:
#preds = reg.predict(X_less_toxic).squeeze().tolist()
#p1 = predictor.predict(X_less_toxic)
p3 = reg.predict(X_less_toxic).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p4 = reg.predict(X_more_toxic).squeeze().tolist()


In [ ]:
# Validation Accuracy
compare(p3,p4)

In [ ]:
scored_sub=reg.predict(X_comments_to_score).squeeze().tolist()
comments_to_score['score2_rank']=rankdata( scored_sub, method='ordinal') 
comments_to_score['score2_df_agregat']=scored_sub

In [ ]:
df1 = pd.read_csv("../input/jigsaw-score-augumentation/jigsaw_train_hate_annotationprob.csv",index_col=0)
if test =='test':
    df1=df1.sample(frac=sample)

In [ ]:
vectorizor = TfidfVectorizer(analyzer='char_wb', max_df=0.8, min_df=1, ngram_range=(1, 3) )

In [ ]:
%%time
X = vectorizor.fit_transform(df1['text'])
temp_score = df1["proposed_score"].values
y=np.around ( temp_score ,decimals = 2)

In [ ]:
%%time
regressor=Ridge(alpha=0.7)
regressor.fit(X, y)

In [ ]:
comments_to_score_set=vectorizor.transform(comments_to_score['text'])
temp_score=regressor.predict(comments_to_score_set)
comments_to_score['df_p0_score1_rank']=rankdata( temp_score, method='ordinal')
comments_to_score['df_p0_score1']=temp_score
comments_to_score.head()

In [ ]:
#comments_to_score_set=vectorizor.transform(comments_to_score['text'])
#preds = reg.predict(X_less_toxic).squeeze().tolist()
#p1 = predictor.predict(X_less_toxic)
p11 = regressor.predict(vectorizor.transform(X_less_toxic)).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p12 = regressor.predict(vectorizor.transform(X_more_toxic)).squeeze().tolist()
# Validation Accuracy
compare(p11,p12)

In [ ]:
vectorizor = TfidfVectorizer(analyzer='char_wb', max_df=0.6, min_df=1, ngram_range=(1, 4) )

In [ ]:
%%time
X = vectorizor.fit_transform(df1['text'])
temp_score = df1["proposed_score2"].values
y=np.around ( temp_score ,decimals = 2)

In [ ]:
%%time
regressor=Ridge(alpha=0.7)
regressor.fit(X, y)

In [ ]:
comments_to_score_set=vectorizor.transform(comments_to_score['text'])
temp_score=regressor.predict(comments_to_score_set)
comments_to_score['df_p2_score1_rank']=rankdata( temp_score, method='ordinal')
comments_to_score['df_p2_score1']=temp_score
comments_to_score.head()

In [ ]:
#comments_to_score_set=vectorizor.transform(comments_to_score['text'])
#preds = reg.predict(X_less_toxic).squeeze().tolist()
#p1 = predictor.predict(X_less_toxic)
p11 = regressor.predict(vectorizor.transform(X_less_toxic)).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p12 = regressor.predict(vectorizor.transform(X_more_toxic)).squeeze().tolist()
# Validation Accuracy
compare(p11,p12)

In [ ]:
vectorizor = TfidfVectorizer(analyzer='char_wb', max_df=0.6, min_df=0.1, ngram_range=(1, 4) )

In [ ]:
%%time
X = vectorizor.fit_transform(df1['text'])
temp_score = df1["proposed_score3"].values
y=np.around ( temp_score ,decimals = 2)

In [ ]:
%%time
regressor=Ridge(alpha=0.7)
regressor.fit(X, y)

In [ ]:
comments_to_score_set=vectorizor.transform(comments_to_score['text'])
temp_score=regressor.predict(comments_to_score_set)
comments_to_score['df_p3_score1_rank']=rankdata( temp_score, method='ordinal')
comments_to_score['df_p3_score1']=temp_score
comments_to_score.head()

In [ ]:
#comments_to_score_set=vectorizor.transform(comments_to_score['text'])
#preds = reg.predict(X_less_toxic).squeeze().tolist()
#p1 = predictor.predict(X_less_toxic)
p13 = regressor.predict(vectorizor.transform(X_less_toxic)).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p14 = regressor.predict(vectorizor.transform(X_more_toxic)).squeeze().tolist()
# Validation Accuracy
compare(p13,p14)

Ruddit ponderation

In [ ]:
ruddit_df.head()

In [ ]:
vectorizor = TfidfVectorizer(analyzer='char_wb', max_df=0.8, min_df=1, ngram_range=(2, 5) )

In [ ]:
%%time
X = vectorizor.fit_transform(ruddit_df['text'])
temp_score = ruddit_df["target"].values
y=np.around ( temp_score ,decimals = 3)

In [ ]:
%%time
regressor=Ridge(alpha=0.5)
regressor.fit(X, y)

In [ ]:
comments_to_score_set=vectorizor.transform(comments_to_score['text'])
temp_score=regressor.predict(comments_to_score_set)
comments_to_score['ruddit_score1_rank']=rankdata( temp_score, method='ordinal')
comments_to_score['ruddit_score1']=temp_score
comments_to_score.head()

In [ ]:
#comments_to_score_set=vectorizor.transform(comments_to_score['text'])
#preds = reg.predict(X_less_toxic).squeeze().tolist()
#p1 = predictor.predict(X_less_toxic)
p5 = regressor.predict(vectorizor.transform(X_less_toxic)).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p6 = regressor.predict(vectorizor.transform(X_more_toxic)).squeeze().tolist()

In [ ]:
# Validation Accuracy
compare(p5,p6)

Fake dataset

In [ ]:
%%time
X = vectorizor.fit_transform(mermeleala_df['text'])
temp_score = mermeleala_df["target"].values
y=np.around ( temp_score ,decimals = 3)

In [ ]:
%%time
regressor=Ridge(alpha=0.5)
regressor.fit(X, y)

In [ ]:
#comments_to_score_set=vectorizor.transform(comments_to_score['text'])
#preds = reg.predict(X_less_toxic).squeeze().tolist()
#p1 = predictor.predict(X_less_toxic)
p7 = regressor.predict(vectorizor.transform(X_less_toxic)).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p8 = regressor.predict(vectorizor.transform(X_more_toxic)).squeeze().tolist()

In [ ]:
# Validation Accuracy
compare(p5,p6)

In [ ]:
comments_to_score_set=vectorizor.transform(comments_to_score['text'])
temp_score=regressor.predict(comments_to_score_set)
comments_to_score['aggregated_score2_rank']=rankdata( temp_score, method='ordinal')
comments_to_score['aggregated_score2']=temp_score
comments_to_score.head()

Another score

In [ ]:
%%time
X = vectorizor.fit_transform(df['text'])
temp_score = df["target"].values
y=np.around ( temp_score ,decimals = 3)

In [ ]:
%%time
regressor=Ridge(alpha=0.5)
regressor.fit(X, y)

In [ ]:
#comments_to_score_set=vectorizor.transform(comments_to_score['text'])
#preds = reg.predict(X_less_toxic).squeeze().tolist()
#p1 = predictor.predict(X_less_toxic)
p9 = regressor.predict(vectorizor.transform(X_less_toxic)).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p10 = regressor.predict(vectorizor.transform(X_more_toxic)).squeeze().tolist()

In [ ]:
# Validation Accuracy
compare(p7,p8)

In [ ]:
comments_to_score_set=vectorizor.transform(comments_to_score['text'])
temp_score=regressor.predict(comments_to_score_set)
comments_to_score['df_score2_rank']=rankdata( temp_score, method='ordinal')
comments_to_score['dfscore2']=temp_score
comments_to_score.head()

ofensiveness

In [ ]:
df.head()

In [ ]:
extend_df = pd.read_csv("../input/jigsaw-score-augumentation/jigsaw_train_hate_annotationprob.csv",index_col=0)
if test =='test':
    extend_df=extend_df.sample(frac=sample)

In [ ]:
%%time
X = vectorizor.fit_transform(extend_df['text'])
temp_score = extend_df["offensive_agreement_rating"].values
y=np.around ( temp_score ,decimals = 3)

In [ ]:
%%time
regressor=Ridge(alpha=0.5)
regressor.fit(X, y)

In [ ]:
#comments_to_score_set=vectorizor.transform(comments_to_score['text'])
#preds = reg.predict(X_less_toxic).squeeze().tolist()
#p1 = predictor.predict(X_less_toxic)
p11 = regressor.predict(vectorizor.transform(X_less_toxic)).squeeze().tolist()
#p2 = predictor.predict(X_more_toxic)
p12 = regressor.predict(vectorizor.transform(X_more_toxic)).squeeze().tolist()

In [ ]:
# Validation Accuracy
compare(p9,p10)

In [ ]:
comments_to_score_set=vectorizor.transform(comments_to_score['text'])
temp_score=regressor.predict(comments_to_score_set)
comments_to_score['df_score2_1rank']=rankdata( temp_score, method='ordinal')
comments_to_score['dfscore2_1']=temp_score
comments_to_score.head()

In [ ]:
comments_to_score.head(50)

In [ ]:
comments_to_score.columns

In [ ]:
comments_to_score['grand_slam']=comments_to_score['score1_rank']+comments_to_score['score2_rank']+comments_to_score['ruddit_score1_rank']+comments_to_score['aggregated_score2_rank']+comments_to_score['df_score2_rank']+comments_to_score['df_score2_1rank']+comments_to_score['df_p0_score1_rank']+comments_to_score['df_p2_score1_rank']+comments_to_score['df_p3_score1_rank']

In [ ]:
comments_to_score['grand_slam_ranks']=rankdata( comments_to_score['grand_slam'], method='ordinal')

In [ ]:
#comments_to_score['score']=scored_sub

In [ ]:
submission=comments_to_score.loc[:,['grand_slam_ranks']]


In [ ]:
submission.rename(columns={"grand_slam_ranks": 'score'}, inplace = True)

In [ ]:
submission.to_csv('submission.csv')

In [ ]:
submission.head()

Ruddit -dumb test

In [ ]:
comments_to_score.to_csv('/kaggle/working/comments_to_score.csv')

In [ ]:
df_test.head()

In [ ]:
df_test['p1']=p1
df_test['p2']=p2
df_test['p3']=p3
df_test['p4']=p4
df_test['p5']=p5
df_test['p6']=p6
df_test['p7']=p7
df_test['p8']=p8
df_test['p9']=p9
df_test['p10']=p10

In [ ]:
(df_test['p2']-df_test['p1']).sum()

In [ ]:
(df_test['p4']-df_test['p3']).sum()

In [ ]:
(df_test['p6']-df_test['p5']).sum()

In [ ]:
(df_test['p8']-df_test['p7']).sum()

In [ ]:
(df_test['p10']-df_test['p9']).sum()

In [ ]:
# submission=comments_to_score[['score']]

In [ ]:
# submission.to_csv('submission.csv')